# Transitions in Centralities
Comparison of centralities transitions in the Spotify graph and in the SGC model

## Configure

Set the dataset base path

In [ ]:
spotify_basename = "graphs/spotify-2018"

Set the random graph base path

In [ ]:
sgc_basename = "graphs/sgc"

Set the centralities CSV path

In [ ]:
csv_path = "./centralities_comparison.csv"

Set `jvm_path` to your java virtual machine full path

In [ ]:
jvm_path = None

Set up logging

In [ ]:
import logging
logging.basicConfig(
  level=logging.INFO,
  format='%(asctime)s %(name)-12s %(levelname)-8s: %(message)s',
  datefmt='%Y-%m-%d %H:%M:%S',
)

Start jvm

In [ ]:
from featgraph.jwebgraph import start_jvm
start_jvm(jvm_path=jvm_path)

## Load the Spotify graph

In [ ]:
import featgraph.jwebgraph.utils
from featgraph import jwebgraph
import os

graph = jwebgraph.utils.BVGraph(spotify_basename)
print(graph)

for r in (
  "graph", "properties", "ids.txt"
):
  if not os.path.isfile(graph.path(r)):
    raise FileNotFoundError(graph.path(r))

## Sample a graph from the SGC model

In [ ]:
from featgraph import sgc, pathutils

seed = 42
sgc_model = sgc.SGCModel()
sgc_graph = jwebgraph.utils.BVGraph(sgc_basename)
if pathutils.notisglob(sgc_graph.path("*")):
  logging.info("Sampling SGC graph")
  sgc_nxgraph = sgc_model(seed=seed)
  logging.info("Converting nxgraph to BVGraph")
  sgc.to_bv(sgc_nxgraph, sgc_basename)

## Generating subgraphs for different popularity thresholds

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

def thresh_graphpath(t: float, g) -> str:
  return g.path(f"popularity-{t:02.0f}")

def thresh_graphmap(t: float, g, mv=-20):
  return map(lambda p: p > t, g.popularity(mv))

thresh = np.arange(0, 81, 5)

for t in tqdm(thresh):
  logging.info("Thresholding graph at popularity > %f", t)
  for g in (graph, sgc_graph):
    path_gt = thresh_graphpath(t, g)
    if pathutils.notisglob(path_gt + ".*"):
      logging.info("Computing: %s", path_gt)
      g.transform_map(path_gt, thresh_graphmap(t, g))

### Compute centralities

In [ ]:
for t in tqdm(thresh):
  for g in (graph, sgc_graph):
    subg = jwebgraph.utils.BVGraph(thresh_graphpath(t, g))
    subg.compute_transpose()
    subg.compute_degrees()
    subg.compute_harmonicc()
    subg.compute_pagerank()
    subg.compute_closenessc()

### Build centralities dataframe

In [ ]:
import pandas as pd
import itertools

basegraphs = {
  "spotify": (
    graph, "supergenre",
    ("classical", "hip-hop", "rock"),
    lambda g: (lambda x: g in x),
  ),
  "sgc": (
    sgc_graph, "type_sgc",
    ("celebrities", "community leaders", "masses"),
    lambda g: (lambda x: x == g),
  ),
}
centralities = ("indegrees", "harmonicc", "pagerank", "closenessc")

data = {
  k: [] for k in (
    "graph",
    "class_type",
    "class_value",
    "threshold",
    "centrality",
    "mean",
    "std",
    "quartile-1",
    "median",
    "quartile-3",
    "nnodes",
    "narcs",
  )
}

if pathutils.notisfile(csv_path):
  for (k, (baseg, gen_k, gen_vals, check_fn)), t, c in tqdm(
    list(itertools.product(
        basegraphs.items(), thresh, centralities
      )
    )
  ):
    for gen in gen_vals:
      data["graph"].append(k)
      data["class_type"].append(gen_k)
      data["class_value"].append(gen)
      data["threshold"].append(t)
      data["centrality"].append(c)

      g = jwebgraph.utils.BVGraph(thresh_graphpath(t, baseg))
      data["nnodes"].append(g.numNodes())
      data["narcs"].append(g.numNodes())
      
      values = np.array(list(itertools.compress(
        getattr(g, c)(),
        map(check_fn(gen), getattr(g, gen_k)())
      )))
      data["mean"].append(np.mean(values))
      data["std"].append(np.std(values))
      for qk, qv in zip(
        (
          "quartile-1",
          "median",
          "quartile-3",
        ),
        np.quantile(
          values,
          (0.25, 0.5 , 0.75)
        )
      ):
        data[qk].append(qv)
      del values

  df = pd.DataFrame(data=data)
  df.to_csv(csv_path)
else:
  df = pd.read_csv(csv_path, index_col=0)
df["nnodes_inv"] = 1 / df["nnodes"]
df["narcs_inv"] = 1 / df["narcs"]
df

## Plot centrality transitions

In [ ]:
# prepare plots
from matplotlib import pyplot as plt
import matplotlib as mpl
_default_cols = mpl.rcParams["axes.prop_cycle"].by_key()["color"]

_gen_cols = {
  "classical": _default_cols[0],
  "hip-hop": _default_cols[1],
  "rock": _default_cols[2],
  "community leaders": _default_cols[0],
  "celebrities": _default_cols[1],
  "masses": _default_cols[2],
}

def plot_centrality_comparison(
  centrality_name: str,
  norm = None,
  logy: bool = False,
  save: bool = False,
  aspect: float = 8/9,
  width: float = 12,
  figext: str = "svg",
  gen_cols = _gen_cols,
  std_scale: float = 0.7,
  fill_alpha: float = 0.1,
  median: bool = True,
):
  fig, ax = plt.subplots(2, 1, sharex=True)
  for a, graph_name in zip(ax, ("spotify", "sgc")):
    dff = df[
      (df["graph"] == graph_name) & (df["centrality"] == centrality_name)
    ]
    for k in set(dff["class_value"]):
      dffk = dff[dff["class_value"] == k]
      kx = dffk["threshold"].to_numpy()
      idx = np.argsort(kx)
      kx = kx[idx]
      
      if median:
        kq1 = dffk["quartile-1"].to_numpy()[idx]
        kq2 = dffk["median"].to_numpy()[idx]
        kq3 = dffk["quartile-3"].to_numpy()[idx]
      else:
        kq2 = dffk["mean"].to_numpy()[idx]
        ks = dffk["mean"].to_numpy()[idx] * std_scale
        kq1 = kq2 - std_scale
        kq3 = kq2 + std_scale
      if norm:
        kn = dffk[norm].to_numpy()[idx]
        kq1 /= kn
        kq2 /= kn
        kq3 /= kn
      a.plot(
        kx, kq2,
        label=k,
        c=gen_cols.get(k, "k")
      )
      a.fill_between(
        kx,
        kq1,
        kq3,
        facecolor=gen_cols.get(k, "k"),
        alpha=fill_alpha,
      )
    if logy:
      a.set_yscale("log")
    a.legend()
    a.set_title(graph_name)
    a.set_ylabel(centrality_name)
  a.set_xlabel("popularity threshold")
  fig.set_size_inches(width * np.array([aspect, 1]))
  if save:
    plt.savefig(
      f"compare-{centrality_name}" + \
      (f"-norm_{norm}" if norm else "") + \
      (f"-median" if logy else "") + \
      (f"-semilogy" if logy else "") + \
      f".{figext}",
      bbox_inches="tight",
    )

In [ ]:
import seaborn as sns

sns.set()
plot_centrality_comparison(
  centrality_name="pagerank",
  norm="nnodes_inv",
  logy=True,
)

## Plot boxplots

In [ ]:
# prepare plots
from IPython.display import SVG, display
import pygal

def plot_centrality_boxes(
  c: str,
  t: float,
  basegraph: str = "spotify",
  savefig: bool = False,
  figext: str = "svg"
):
  box_plot = pygal.Box()
  box_plot.title = f"{c}\npopularity > {t:.0f}\n{basegraph}"
  
  baseg, gen_k, gen_vals, check_fn = basegraphs[basegraph]
  g = jwebgraph.utils.BVGraph(thresh_graphpath(t, baseg))
  for gen in gen_vals:
    box_plot.add(gen, np.array(list(itertools.compress(
      getattr(g, c)(),
      map(check_fn(gen), getattr(g, gen_k)())
    ))))
  
  display(SVG(box_plot.render()))
  if savefig:
    box_plot.render_to_file(
      f"boxplot-{basegraph}-{c}.{figext}"
    )

In [ ]:
plot_centrality_boxes(
  "harmonicc", 50,
  "spotify",
)